Романовский А.Н. 3.10.2022

В данной задаче нам представлены данные в которых ключевое поле — это то, просрочит ли клиент микро-финансовой организации выплату более чем на 60 дней или нет. 
Нам нужно построить модель, которая по входным данным предсказывала бы с максимальной точностью ключе-вой параметр. 
Основной параметр это delinq60plus. Остальные парамет-ры интуитивно понятны из таблиц.
# Цель

Построить модель машинного обучения для предсказания вероятности того, что клиент уйдет в просрочку более 60 дней.

# Задача
осуществить корректную загрузку данных
осуществить разведывательный анализ данных:
- разбить выборку на обучающую и тестовую
- проанализировать пропуски и решить, что с ними делать
- проанализировать выбросы
- создать/ удалить переменные
- закодировать категориальные переменные
- нормализовать числовые переменные (при необходимости)

In [3]:
import pandas as pd
import numpy as np
import sqlite3
import seaborn as sns

In [4]:
con = sqlite3.connect('forsqltask')

In [5]:
df = pd.read_csv('MFOcredit.csv', sep = ';', )                              

In [171]:
len(df) - len(df.drop_duplicates(keep = 'first'))  # проверка дубликатов

0

In [172]:
df['date_start'] = pd.to_datetime(df['date_start'])
df['date_end'] = pd.to_datetime(df['date_end']) # меняю формат даты

In [8]:
df.head(2)

,id,date_start,date_end,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus
0,1,2013-01-03,2013-01-12,Мужской,44,Нет,Собственное,Гражданский брак/женат/замужем,Нет,Официальное,Новосибирская область,Нет,Да,Нет
1,2,2013-01-03,2013-01-17,Мужской,21,Пропуск поля,Живут с родителями,Холост,Нет,Официальное,Кемеровская область юг,Да,Нет,Нет


In [9]:
df.to_sql('MFOcredit' ,con,index=False,if_exists='replace')

35212

In [10]:
def select(sql):
    return pd.read_sql(sql,con)

In [11]:
sql = '''Select *
FROM MFOcredit '''

In [12]:
select(sql).head(2)

,id,date_start,date_end,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus
0,1,2013-01-03 00:00:00,2013-01-12 00:00:00,Мужской,44,Нет,Собственное,Гражданский брак/женат/замужем,Нет,Официальное,Новосибирская область,Нет,Да,Нет
1,2,2013-01-03 00:00:00,2013-01-17 00:00:00,Мужской,21,Пропуск поля,Живут с родителями,Холост,Нет,Официальное,Кемеровская область юг,Да,Нет,Нет


In [11]:
sql = '''PRAGMA table_info('MFOcredit')'''

In [12]:
select(sql)

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,0
1,1,date_start,TIMESTAMP,0,None,0
2,2,date_end,TIMESTAMP,0,None,0
3,3,gender,TEXT,0,None,0
4,4,age,INTEGER,0,None,0
5,5,auto,TEXT,0,None,0
6,6,housing,TEXT,0,None,0
7,7,marstatus,TEXT,0,None,0
8,8,regclient,TEXT,0,None,0
9,9,jobtype,TEXT,0,None,0


#### Название столбцов
* id - номер клиента 
* date_start 	- дата начала займа 
* date_end  - дата завершение займа 
* gender 	- пол
* age - возраст заемщика
* auto - наличие автомабиля у заемщика
* housing  - информация о жилье 
* marstatus - семейное положение
* regclient - флаг регулярности заемщика 	
* jobtype - наличие офоциальной работы
* region - область проживания 
* credits - наличие кредита	
* children - наличие детей 
* delinq60plus - целевой флаг (просрочка кредита более 60 дней)
____



In [16]:
sql = '''SELECT auto, COUNT(auto)
FROM MFOcredit
GROUP BY auto'''
select(sql)

,auto,COUNT(auto)
0,Да,3670
1,Нет,17465
2,Пропуск поля,14077


In [17]:
sql = '''SELECT housing,COUNT(housing)
FROM MFOcredit
GROUP BY housing'''
select(sql)

,housing,COUNT(housing)
0,Арендуемое,2020
1,Долевая собственность,2955
2,Живут с родителями,3177
3,Муниципальное,1358
4,Пропуск поля,7603
5,Собственное,18099


In [19]:
sql = '''SELECT jobtype,COUNT(jobtype)
FROM MFOcredit
GROUP BY jobtype'''
select(sql)

,jobtype,COUNT(jobtype)
0,Неофициальное,1167
1,Официальное,19588
2,Пропуск поля,14457


In [21]:
sql = '''SELECT marstatus,COUNT(marstatus)
FROM MFOcredit
GROUP BY marstatus'''
select(sql)

,marstatus,COUNT(marstatus)
0,Вдова/вдовец,1845
1,Гражданский брак/женат/замужем,14099
2,Пропуск поля,7564
3,Разведен,2871
4,Холост,8833


In [22]:
sql = '''SELECT credits,COUNT(credits)
FROM MFOcredit
GROUP BY credits'''
select(sql)

,credits,COUNT(credits)
0,Да,16140
1,Нет,18126
2,Пропуск поля,946


In [23]:
sql = '''SELECT children,COUNT(children)
FROM MFOcredit
GROUP BY children'''
select(sql)

,children,COUNT(children)
0,Да,13326
1,Нет,21141
2,Пропуск поля,745


#### Есть пропуски:
* auto - 14077
* housing - 7603
* jobtype - 14457
* marstatus - 7564
* credits - 946
* children - 745

In [173]:
sql = '''ALTER TABLE MFOcredit 
ADD credit_period as (julianday(date_end) - julianday(date_start))'''
con.execute(sql) # добавляю столбец с периодом займа в днях

In [25]:
sql = '''select * from MFOcredit'''
select(sql).head(1)

,id,date_start,date_end,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus
0,1,2013-01-03 00:00:00,2013-01-12 00:00:00,Мужской,44,Нет,Собственное,Гражданский брак/женат/замужем,Нет,Официальное,Новосибирская область,Нет,Да,Нет


In [24]:
sql = '''
create table test as
SELECT   *
FROM MFOcredit 
ORDER BY Random()
LIMIT (select count(*)/(100) * 30 from MFOcredit )'''
con.execute(sql) #создаю таблицу с тестовой выборкой 

In [26]:
sql = '''select * from test '''
select(sql).head(2)

,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus,credit_period
0,Женский,39,Пропуск поля,Пропуск поля,Пропуск поля,Да,Пропуск поля,Кемеровская область юг,Нет,Нет,Да,14.0
1,Мужской,32,Пропуск поля,Живут с родителями,Холост,Да,Официальное,Кемеровская область север,Нет,Нет,Да,14.0


In [34]:
sql = '''
create table train as
select * from MFOcredit 
where not exists(select * from test
where MFOcredit.id = test.id  )'''
#создаю таблицу с тренировочной выборкой 

In [ ]:
con.execute(sql)

In [15]:
sql = '''select * from train '''
select(sql).head(2)

,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus,credit_period
0,Мужской,44,Нет,Собственное,Гражданский брак/женат/замужем,Нет,Официальное,Новосибирская область,Нет,Да,Нет,9.0
1,Мужской,21,Пропуск поля,Живут с родителями,Холост,Нет,Официальное,Кемеровская область юг,Да,Нет,Нет,14.0


In [ ]:
sql = '''ALTER TABLE test
DROP COLUMN id
'''
con.execute(sql)#удаляю столбец с id клиента 

In [ ]:
sql = '''ALTER TABLE test
DROP COLUMN date_start
'''
con.execute(sql) #удаляю столбцы с датами займа 

In [ ]:
sql = '''ALTER TABLE test
DROP COLUMN date_end
'''
con.execute(sql)

In [16]:
sql = '''select * from test as t'''
select(sql).head(2)

,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus,credit_period
0,Женский,39,Пропуск поля,Пропуск поля,Пропуск поля,Да,Пропуск поля,Кемеровская область юг,Нет,Нет,Да,14.0
1,Мужской,32,Пропуск поля,Живут с родителями,Холост,Да,Официальное,Кемеровская область север,Нет,Нет,Да,14.0


In [ ]:
sql = '''ALTER TABLE train 
DROP COLUMN id
'''
con.execute(sql)

In [ ]:
sql = '''ALTER TABLE train 
DROP COLUMN date_start
'''
con.execute(sql)

In [ ]:
sql = '''ALTER TABLE train 
DROP COLUMN date_end
'''
con.execute(sql)

In [18]:
sql = '''select * from train'''
select(sql).head(2)

,gender,age,auto,housing,marstatus,regclient,jobtype,region,credits,children,delinq60plus,credit_period
0,Мужской,44,Нет,Собственное,Гражданский брак/женат/замужем,Нет,Официальное,Новосибирская область,Нет,Да,Нет,9.0
1,Мужской,21,Пропуск поля,Живут с родителями,Холост,Нет,Официальное,Кемеровская область юг,Да,Нет,Нет,14.0


In [19]:
sql1= '''SELECT  auto, gender, age, count(auto) 
from MFOcredit
where gender  = 'Мужской'
and (auto = 'Да')
group by age
'''
select(sql1).head(2)

,auto,gender,age,count(auto)
0,Да,Мужской,18,2
1,Да,Мужской,19,19


In [20]:
sql2= '''SELECT  auto, gender, age, count(auto) 
from MFOcredit
where gender  = 'Мужской'
and (auto = 'Нет')
group by age
'''
select(sql2).head(2)

,auto,gender,age,count(auto)
0,Нет,Мужской,18,3
1,Нет,Мужской,19,136


In [21]:
age_auto = pd.DataFrame((select(sql1)['count(auto)'] - select(sql2)['count(auto)']), (select(sql2)['age']))
age_auto.head(3)

,count(auto)
age,
18,-94.0
19,-106.0
20,-110.0


In [22]:
sql1= '''SELECT  auto, gender, age, count(auto) 
from MFOcredit
where gender  = 'Женский'
and (auto = 'Да')
group by age
'''
select(sql1).head(2)

,auto,gender,age,count(auto)
0,Да,Женский,19,5
1,Да,Женский,20,7


In [23]:
sql2= '''SELECT  auto, gender, age, count(auto) 
from MFOcredit
where gender  = 'Женский'
and (auto = 'Нет')
group by age
'''
select(sql2).head(2)

,auto,gender,age,count(auto)
0,Нет,Женский,18,1
1,Нет,Женский,19,116


In [24]:
age_auto = pd.DataFrame((select(sql1)['count(auto)'] - select(sql2)['count(auto)']), (select(sql2)['age']))
age_auto.head(3)

,count(auto)
age,
18,-204.0
19,-214.0
20,-163.0


* Сгрупировав данные о наличии авто, мужского и женского пола по возрастам, посчиатв разницу между флагом "Да" и "Нет", везеде получаем отрицательное число, таким образов делаю вывод что, у большинства авто все же нет, и данные пропуски заполняю флагом "Нет"
_____

In [25]:
sql = '''UPDATE test 
SET auto = 'Нет' 
WHERE auto = "Пропуск поля" '''
con.execute(sql)

In [26]:
sql = '''UPDATE train 
SET auto = 'Нет' 
WHERE auto = "Пропуск поля" '''
con.execute(sql)

In [27]:
sql = '''SELECT housing,gender, age,COUNT(housing)
FROM MFOcredit
where gender = 'Женский'
group by housing'''
select(sql)

,housing,gender,age,COUNT(housing)
0,Арендуемое,Женский,27,785
1,Долевая собственность,Женский,30,1329
2,Живут с родителями,Женский,20,1162
3,Муниципальное,Женский,51,836
4,Пропуск поля,Женский,27,3801
5,Собственное,Женский,47,10362


In [28]:
sql = '''SELECT housing,gender, age,COUNT(housing)
FROM MFOcredit
where gender = 'Мужской'
group by housing'''
select(sql)

,housing,gender,age,COUNT(housing)
0,Арендуемое,Мужской,22,1235
1,Долевая собственность,Мужской,51,1626
2,Живут с родителями,Мужской,21,2015
3,Муниципальное,Мужской,20,522
4,Пропуск поля,Мужской,25,3802
5,Собственное,Мужской,44,7737


* Исходя из последних двух таблиц, пропуски в столбце housing буду заполнять связывая с возрастными диапазонами
* заполнение стобца  housing для test части

In [29]:
sql = '''UPDATE test 
SET housing = 'Живут с родителями' 
WHERE age between '0' and "20" and housing = "Пропуск поля" and gender = 'Женский'  
'''
con.execute(sql)

In [30]:
sql = '''UPDATE test 
SET housing = 'Арендуемое' 
WHERE age between '20' and "27" and housing = "Пропуск поля"  and gender = 'Женский' 
'''
con.execute(sql)

In [31]:
sql = '''UPDATE test 
SET housing = 'Долевая собственность' 
WHERE age between '27' and "30" and housing = "Пропуск поля" and gender = 'Женский'  
'''
con.execute(sql)

In [32]:
sql = '''UPDATE test 
SET housing = 'Собственное' 
WHERE age between '30' and "72" and housing = "Пропуск поля" and gender = 'Женский'  
'''
con.execute(sql)

In [33]:
sql = '''UPDATE test 
SET housing = 'Живут с родителями' 
WHERE age between '0' and "21" and housing = "Пропуск поля" and gender = 'Мужской' 
'''
con.execute(sql)

In [34]:
sql = '''UPDATE test 
SET housing = 'Арендуемое' 
WHERE age between '21' and "25" and housing = "Пропуск поля" and gender = 'Мужской' 
'''
con.execute(sql)

In [35]:
sql = '''UPDATE test 
SET housing = 'Собственное' 
WHERE age between '25' and "51" and housing = "Пропуск поля" and gender = 'Мужской' 
'''
con.execute(sql)

In [36]:
sql = '''UPDATE test 
SET housing = 'Долевая собственность' 
WHERE age between '51' and "74" and housing = "Пропуск поля" and gender = 'Мужской' 
'''
con.execute(sql)

In [37]:
sql = '''SELECT housing,gender,COUNT(housing)
FROM test
where housing = 'Пропуск поля'

group by age'''
select(sql)

,housing,gender,COUNT(housing)


* заполнение стобца  housing для train части

In [38]:
sql = '''UPDATE train 
SET housing = 'Живут с родителями' 
WHERE age between '0' and "20" and housing = "Пропуск поля" and gender = 'Женский'  
'''
con.execute(sql)

In [39]:
sql = '''UPDATE train 
SET housing = 'Арендуемое' 
WHERE age between '20' and "27" and housing = "Пропуск поля"  and gender = 'Женский' 
'''
con.execute(sql)

In [40]:
sql = '''UPDATE train 
SET housing = 'Долевая собственность' 
WHERE age between '27' and "30" and housing = "Пропуск поля" and gender = 'Женский'  
'''
con.execute(sql)

In [41]:
sql = '''UPDATE train 
SET housing = 'Собственное' 
WHERE age between '30' and "75" and housing = "Пропуск поля" and gender = 'Женский'  
'''
con.execute(sql)

In [42]:
sql = '''UPDATE train 
SET housing = 'Живут с родителями' 
WHERE age between '0' and "21" and housing = "Пропуск поля" and gender = 'Мужской' 
'''
con.execute(sql)

In [43]:
sql = '''UPDATE train 
SET housing = 'Арендуемое' 
WHERE age between '21' and "25" and housing = "Пропуск поля" and gender = 'Мужской' 
'''
con.execute(sql)

In [44]:
sql = '''UPDATE train 
SET housing = 'Собственное' 
WHERE age between '25' and "51" and housing = "Пропуск поля" and gender = 'Мужской' 
'''
con.execute(sql)

In [45]:
sql = '''UPDATE train 
SET housing = 'Долевая собственность' 
WHERE age between '51' and "75" and housing = "Пропуск поля" and gender = 'Мужской' 
'''
con.execute(sql)

* Заполнение пропусков в столбце jobtype

In [46]:
sql = '''SELECT jobtype,gender,age, COUNT(jobtype)
FROM MFOcredit
where gender = "Женский"
group by age'''
select(sql).head(1)

,jobtype,gender,age,COUNT(jobtype)
0,Официальное,Женский,18,1


In [47]:
sql = '''SELECT jobtype,gender,age, COUNT(jobtype)
FROM MFOcredit
where gender = "Мужской"
group by age'''
select(sql).head(1)

,jobtype,gender,age,COUNT(jobtype)
0,Официальное,Мужской,18,5


* Незаметив каких-то зависимостей от возраста, заменяю пропуски частым по столбцу "Официальное"

In [48]:
sql = '''UPDATE test 
SET jobtype = 'Официальное' 
WHERE  jobtype = 'Пропуск поля'
'''
con.execute(sql)

In [49]:
sql = '''UPDATE train 
SET jobtype = 'Официальное' 
WHERE  jobtype = 'Пропуск поля'
'''
con.execute(sql)

* Заполнение пропусков в столбце marstatus, заполняю аналогично возрастными диапазонами 

In [50]:
sql = '''SELECT marstatus,gender,age, COUNT(jobtype)
FROM MFOcredit
where gender = "Мужской"
group by marstatus'''
select(sql)

,marstatus,gender,age,COUNT(jobtype)
0,Вдова/вдовец,Мужской,50,166
1,Гражданский брак/женат/замужем,Мужской,44,7122
2,Пропуск поля,Мужской,36,3776
3,Разведен,Мужской,40,931
4,Холост,Мужской,21,4942


In [51]:
sql = '''SELECT marstatus,gender,age, COUNT(jobtype)
FROM MFOcredit
where gender = "Женский"
group by marstatus'''
select(sql)

,marstatus,gender,age,COUNT(jobtype)
0,Вдова/вдовец,Женский,51,1679
1,Гражданский брак/женат/замужем,Женский,47,6977
2,Пропуск поля,Женский,63,3788
3,Разведен,Женский,29,1940
4,Холост,Женский,39,3891


* для части test

In [52]:
sql = '''UPDATE test 
SET marstatus = 'Холост' 
WHERE age between '0' and "36" and marstatus = "Пропуск поля" and gender = 'Мужской' 
'''
con.execute(sql)

In [53]:
sql = '''UPDATE test 
SET marstatus = 'Гражданский брак/женат/замужем' 
WHERE age between '36' and "50" and marstatus = "Пропуск поля" and gender = 'Мужской' 
'''
con.execute(sql)

In [54]:
sql = '''UPDATE test 
SET marstatus = 'Вдова/вдовец' 
WHERE age between '50' and "75" and marstatus = "Пропуск поля" and gender = 'Мужской' 
'''
con.execute(sql)

In [55]:
sql = '''UPDATE test 
SET marstatus = 'Холост' 
WHERE age between '0' and "39" and marstatus = "Пропуск поля" and gender = 'Женский' 
'''
con.execute(sql)

In [56]:
sql = '''UPDATE test 
SET marstatus = 'Гражданский брак/женат/замужем' 
WHERE age between '39' and "51" and marstatus = "Пропуск поля" and gender = 'Женский' 
'''
con.execute(sql)

In [57]:
sql = '''UPDATE test 
SET marstatus = 'Вдова/вдовец' 
WHERE age between '51' and "75" and marstatus = "Пропуск поля" and gender = 'Женский' 
'''
con.execute(sql)

* для части train 

In [58]:
sql = '''UPDATE train  
SET marstatus = 'Холост' 
WHERE age between '0' and "36" and marstatus = "Пропуск поля" and gender = 'Мужской' 
'''
con.execute(sql)

In [59]:
sql = '''UPDATE train  
SET marstatus = 'Гражданский брак/женат/замужем' 
WHERE age between '36' and "50" and marstatus = "Пропуск поля" and gender = 'Мужской' 
'''
con.execute(sql)

In [60]:
sql = '''UPDATE train 
SET marstatus = 'Вдова/вдовец' 
WHERE age between '50' and "75" and marstatus = "Пропуск поля" and gender = 'Мужской' 
'''
con.execute(sql)

In [61]:
sql = '''UPDATE train  
SET marstatus = 'Холост' 
WHERE age between '0' and "39" and marstatus = "Пропуск поля" and gender = 'Женский' 
'''
con.execute(sql)

In [62]:
sql = '''UPDATE train  
SET marstatus = 'Гражданский брак/женат/замужем' 
WHERE age between '39' and "51" and marstatus = "Пропуск поля" and gender = 'Женский' 
'''
con.execute(sql)

In [63]:
sql = '''UPDATE train  
SET marstatus = 'Вдова/вдовец' 
WHERE age between '51' and "75" and marstatus = "Пропуск поля" and gender = 'Женский' 
'''
con.execute(sql)

* Заполненние пропусков в столбце credits. т.к. кол-во пропусков у мужчин меньше и кредитов у них мешньше, решаю заполнить мужчин как "Нет", а женщин "Да"

In [64]:
sql = '''SELECT credits,gender,age, COUNT(credits)
FROM MFOcredit
where gender = "Женский"
group by credits'''
select(sql)

,credits,gender,age,COUNT(credits)
0,Да,Женский,63,8810
1,Нет,Женский,47,8913
2,Пропуск поля,Женский,30,552


In [65]:
sql = '''SELECT credits,gender,age, COUNT(credits)
FROM MFOcredit
where gender = "Мужской"
group by credits'''
select(sql)

,credits,gender,age,COUNT(credits)
0,Да,Мужской,21,7330
1,Нет,Мужской,44,9213
2,Пропуск поля,Мужской,25,394


In [66]:
sql = '''UPDATE test  
SET credits = 'Да' 
WHERE credits = "Пропуск поля" and gender = 'Женский' 
'''
con.execute(sql)

In [67]:
sql = '''UPDATE test  
SET credits = 'Нет' 
WHERE credits = "Пропуск поля" and gender = 'Мужской' 
'''
con.execute(sql)

In [68]:
sql = '''UPDATE train  
SET credits = 'Да' 
WHERE credits = "Пропуск поля" and gender = 'Женский' 
'''
con.execute(sql)

In [69]:
sql = '''UPDATE train  
SET credits = 'Нет' 
WHERE credits = "Пропуск поля" and gender = 'Мужской' 
'''
con.execute(sql)

* Заполнение пропусков в столбце children тоже по возрастным диапазонам 

In [70]:
sql = '''SELECT children,COUNT(children)
FROM MFOcredit
GROUP BY children'''
select(sql)

,children,COUNT(children)
0,Да,13326
1,Нет,21141
2,Пропуск поля,745


In [168]:
sql = '''SELECT children,gender,age, COUNT(children)
FROM MFOcredit
where gender = "Мужской"
group by age'''
select(sql).head(3)

,children,gender,age,COUNT(children)
0,Нет,Мужской,18,5
1,Нет,Мужской,19,244
2,Нет,Мужской,20,410


In [169]:
sql = '''SELECT children,gender,age, COUNT(children)
FROM MFOcredit
where gender = "Женский"
group by age'''
select(sql).head(3)

,children,gender,age,COUNT(children)
0,Нет,Женский,18,1
1,Нет,Женский,19,181
2,Нет,Женский,20,252


In [73]:
sql = '''UPDATE test  
SET children = 'Нет' 
WHERE age between '0' and "45" and children = "Пропуск поля" and gender = 'Мужской' 
'''
con.execute(sql)

In [74]:
sql = '''UPDATE test  
SET children = 'Да' 
WHERE age between '45' and "75" and children = "Пропуск поля" and gender = 'Мужской' 
'''
con.execute(sql)

In [75]:
sql = '''UPDATE test  
SET children = 'Нет' 
WHERE age between '0' and "31" and children = "Пропуск поля" and gender = 'Женский' 
'''
con.execute(sql)

In [76]:
sql = '''UPDATE test  
SET children = 'Да' 
WHERE age between '31' and "75" and children = "Пропуск поля" and gender = 'Женский' 
'''
con.execute(sql)

In [77]:
sql = '''UPDATE train  
SET children = 'Нет' 
WHERE age between '0' and "45" and children = "Пропуск поля" and gender = 'Мужской' 
'''
con.execute(sql)

In [78]:
sql = '''UPDATE train  
SET children = 'Да' 
WHERE age between '45' and "75" and children = "Пропуск поля" and gender = 'Мужской' 
'''
con.execute(sql)

In [79]:
sql = '''UPDATE train  
SET children = 'Нет' 
WHERE age between '0' and "31" and children = "Пропуск поля" and gender = 'Женский' 
'''
con.execute(sql)

In [80]:
sql = '''UPDATE train  
SET children = 'Да' 
WHERE age between '31' and "75" and children = "Пропуск поля" and gender = 'Женский' 
'''
con.execute(sql)

* Обработанные данные забираю из БД, отделяю целевую переменную 
* Кодирую переменные 

In [95]:
sql = '''select * from test'''

In [97]:
test_data = select(sql)

In [99]:
y_test_data = test_data['delinq60plus']

In [100]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_test_data = le.fit_transform(y_test_data)

y_test_data

array([0, 0, 0, ..., 0, 1, 0])

In [101]:
test_data.drop(['delinq60plus'], axis=1, inplace=True) 

In [102]:
test_data = pd.concat([test_data, 
                      pd.get_dummies(test_data['gender'], prefix="gender"),
                      pd.get_dummies(test_data['auto'], prefix="auto"),
                      pd.get_dummies(test_data['housing'], prefix="housing"),
                      pd.get_dummies(test_data['marstatus'], prefix="marstatus"),
                      pd.get_dummies(test_data['regclient'], prefix="regclient"),
                      pd.get_dummies(test_data['jobtype'], prefix="jobtype"),
                      pd.get_dummies(test_data['region'], prefix="region"),
                      pd.get_dummies(test_data['credits'], prefix="credits"),
                      pd.get_dummies(test_data['children'], prefix="children")],
                      axis = 1)

In [103]:
test_data.drop(['gender','auto','housing','marstatus', 'regclient','jobtype','region','credits', 'children'], axis=1, inplace=True)

In [106]:
train_data = select(sql)

In [107]:
y_train_data = train_data['delinq60plus']

In [108]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_train_data = le.fit_transform(y_train_data)

y_train_data

array([1, 1, 1, ..., 1, 1, 1])

In [109]:
train_data.drop(['delinq60plus'], axis=1, inplace=True) 

In [110]:
train_data = pd.concat([train_data, 
                      pd.get_dummies(train_data['gender'], prefix="gender"),
                      pd.get_dummies(train_data['auto'], prefix="auto"),
                      pd.get_dummies(train_data['housing'], prefix="housing"),
                      pd.get_dummies(train_data['marstatus'], prefix="marstatus"),
                      pd.get_dummies(train_data['regclient'], prefix="regclient"),
                      pd.get_dummies(train_data['jobtype'], prefix="jobtype"),
                      pd.get_dummies(train_data['region'], prefix="region"),
                      pd.get_dummies(train_data['credits'], prefix="credits"),
                      pd.get_dummies(train_data['children'], prefix="children")],
                      axis = 1)

In [111]:
train_data.drop(['gender','auto','housing','marstatus', 'regclient','jobtype','region','credits', 'children'], axis=1, inplace=True)

In [118]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1)) # скалируем данные 

train_X_scaled = scaler.fit_transform(train_data)
test_X_scaled = scaler.fit_transform(test_data)

In [162]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=10)
rfc.fit(train_data, y_train_data)
rfc.score (test_data, y_test_data)

0.6644491319162745

In [167]:
y_pred = rfc.predict_proba(test_data)[:, 1]
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test_data, y_pred)

0.797336905008773

#### Вывод:
* скалирование ухудшило результат на 0.1, выше результат без скалирования 
* c sql пока достаточно сложно
